In [3]:
import cv2
import numpy as np

def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to enhance contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)

    # Apply adaptive thresholding to separate roof area from the background
    _, thresholded = cv2.threshold(equalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return thresholded

def remove_background(image, mask):
    # Convert mask to 3-channel format
    mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Apply bitwise AND operation to remove background based on the mask
    result = cv2.bitwise_and(image, mask_3d)
    return result

def find_roof_contour(mask, min_area):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = filter_contours(contours, min_area)

    if filtered_contours:
        roof_contour = max(filtered_contours, key=cv2.contourArea)
        return roof_contour
    else:
        return None

def filter_contours(contours, min_area):
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_area:
            # Calculate the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Calculate aspect ratio (width to height ratio) of the contour's bounding rectangle
            aspect_ratio = float(w) / h

            # Filter out contours with aspect ratio greater than a certain threshold
            if aspect_ratio < 5:
                filtered_contours.append(contour)

    return filtered_contours

def calculate_roof_area(roof_contour):
    roof_area = cv2.contourArea(roof_contour)
    return roof_area

def main():
    image_path = 'pmihostel.png'
    min_roof_area = 1000  # Minimum area for considering a contour as a roof (adjust as needed)

    # Load the image
    image = cv2.imread(image_path)

    # Preprocess the image
    preprocessed = preprocess_image(image)

    # Find roof contour
    roof_contour = find_roof_contour(preprocessed, min_roof_area)

    if roof_contour is not None:
        # Calculate roof area
        roof_area = calculate_roof_area(roof_contour)

        # Remove background/
        background_removed = remove_background(image, preprocessed)

        # Print result
        print("Roof Area:", roof_area, "square meters")

        # Optional: Visualize the roof contour and background-removed image
        cv2.drawContours(image, [roof_contour], -1, (0, 255, 0), 2)
        cv2.imshow("Rooftop Image with Contour", image)
        cv2.imshow("Background Removed Image", background_removed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No valid roof contour found.")

if __name__ == '__main__':
    main()


Roof Area: 83378.0 square meters
